# Intro
This notebook lets you run the training and inference on the local EC2 instance instead of the other notebooks that use SageMaker.

# Install requirements

We'll use the same files as the SageMaker training, so we'll first move to the assets directory and run our scripts from there.

In [1]:
%cd /home/ubuntu/environment/FineTuning/HuggingFaceExample/01_finetuning/assets

/home/ubuntu/environment/FineTuning/HuggingFaceExample/01_finetuning/assets


/home/ubuntu/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
%pip install -r requirements.txt


Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Using cached transformers-4.51.3-py3-none-any.whl.metadata (38 kB)
  Using cached numpy-1.25.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
  Using cached tokenizers-0.21.4-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 117.4 MB/s  0:00:00
Using cached numpy-1.25.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 69.2 MB/s  0:00:00
Using cached transformers-4.51.3-py3-none-any.whl (10.4 MB)
Using cached tokenizers-0.21.4-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Training

We will use the same training scripts as we do in the SageMaker examples, we just need to launch them with the torchrun process and the same parameters that we would have passed in.  See the Finetune-TinyLlama-1.1B notebook for more information on the parameters.

Additionally, this example uses a Qwen model.



In [3]:
!torchrun --nnodes 1 --nproc_per_node 2 \
finetune_llama.py \
--bf16 True --dataloader_drop_last True --disable_tqdm True --gradient_accumulation_steps 1 \
--gradient_checkpointing True --learning_rate 5e-05 --logging_steps 10 --lora_alpha 32 \
--lora_dropout 0.05 --lora_r 16 --max_steps 1000 \
--model_id Qwen/Qwen3-1.7B --output_dir ~/environment/ml/qwen \
--per_device_train_batch_size 2 --tensor_parallel_size 2 \
--tokenizer_id Qwen/Qwen3-1.7B

W1004 21:15:25.793000 27737 torch/distributed/run.py:774] 
W1004 21:15:25.793000 27737 torch/distributed/run.py:774] *****************************************
W1004 21:15:25.793000 27737 torch/distributed/run.py:774] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W1004 21:15:25.793000 27737 torch/distributed/run.py:774] *****************************************
Traceback (most recent call last):
  File "/home/ubuntu/environment/FineTuning/HuggingFaceExample/01_finetuning/assets/finetune_llama.py", line 15, in <module>
    from optimum.neuron import NeuronHfArgumentParser as HfArgumentParser
  File "/home/ubuntu/.local/lib/python3.10/site-packages/optimum/neuron/__init__.py", line 19, in <module>
    from .utils.training_utils import patch_transformers_for_neuron_sdk
  File "/home/ubuntu/.local/lib/python3.10/site-packages/opt

# Compilation

Since we have everything installed locally, we don't need to use a training job like on SageMaker.  We can just call the optimum-cli command directly.

The training process runs a merge script at the end, so we are using the output_dir and adding a merged_model path and then saving our compiled model into the compiled_model path.

In [4]:
!optimum-cli export neuron --model /home/ubuntu/environment/ml/qwen/merged_model --task text-generation --sequence_length 512 --batch_size 1 --num_cores 2 /home/ubuntu/environment/ml/qwen/compiled_model


/home/ubuntu/.local/lib/python3.10/site-packages/optimum/commands/env.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution
Traceback (most recent call last):
  File "/home/ubuntu/.local/bin/optimum-cli", line 3, in <module>
    from optimum.commands.optimum_cli import main
  File "/home/ubuntu/.local/lib/python3.10/site-packages/optimum/commands/__init__.py", line 16, in <module>
    from .env import EnvironmentCommand
  File "/home/ubuntu/.local/lib/python3.10/site-packages/optimum/commands/env.py", line 22, in <module>
    from ..neuron.utils import is_neuron_available, is_neuronx_available
  File "/home/ubuntu/.local/lib/python3.10/site-packages/optimum/neuron/__init__.py", line 19, in <module>
    from .utils.training_utils import patch_transfor

# Inference

We will install the Optimum Neuron vllm option.  Then, run inference using the compiled model!

In [5]:
%pip install optimum-neuron[vllm]


Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Using cached scipy-1.15.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached torchvision-0.22.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (6.1 kB)
  Using cached astor-0.8.1-py2.py3-none-any.whl.metadata (4.2 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.6.77-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.6.77-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.6.80-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.5.1.17-py3-none-manylinux_2_28_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.6.4.1-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuff

In [6]:
import os
from vllm import LLM, SamplingParams
llm = LLM(
    model="/home/ubuntu/environment/ml/qwen/compiled_model", #local compiled model
    max_num_seqs=1,
    max_model_len=2048,
    device="neuron",
    tensor_parallel_size=2,
    override_neuron_config={})
example1="""
<|im_start|>system
You are a text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.
SCHEMA:
CREATE TABLE management (department_id VARCHAR); CREATE TABLE department (department_id VARCHAR)<|im_end|>
<|im_start|>user
How many departments are led by heads who are not mentioned?<|im_end|>
<|im_start|>assistant
"""
example2="""
<|im_start|>system
You are a text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.
SCHEMA:
CREATE TABLE courses (course_name VARCHAR, course_id VARCHAR); CREATE TABLE student_course_registrations (student_id VARCHAR, course_id VARCHAR)<|im_end|>
<|im_start|>user
What are the ids of all students for courses and what are the names of those courses?<|im_end|>
<|im_start|>assistant
"""
example3="""
<|im_start|>system
You are a text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.
SCHEMA:
CREATE TABLE table_name_9 (wins INTEGER, year VARCHAR, team VARCHAR, points VARCHAR)<|im_end|>
<|im_start|>user
Which highest wins number had Kawasaki as a team, 95 points, and a year prior to 1981?<|im_end|>
<|im_start|>assistant
"""

prompts = [
    example1,
    example2,
    example3
]

sampling_params = SamplingParams(max_tokens=2048, temperature=0.8)
outputs = llm.generate(prompts, sampling_params)

print("#########################################################")

for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, \n\n Generated text: {generated_text!r} \n")

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 10-04 21:18:16 [__init__.py:39] Available plugins for group vllm.platform_plugins:
INFO 10-04 21:18:16 [__init__.py:41] - optimum_neuron -> optimum.neuron.vllm.plugin:register
INFO 10-04 21:18:16 [__init__.py:44] All plugins in this group will be loaded. Set `VLLM_PLUGINS` to control which plugins to load.
ERROR 10-04 21:18:16 [__init__.py:57] Failed to load plugin optimum_neuron
ERROR 10-04 21:18:16 [__init__.py:57] Traceback (most recent call last):
ERROR 10-04 21:18:16 [__init__.py:57]   File "/home/ubuntu/.local/lib/python3.10/site-packages/vllm/plugins/__init__.py", line 54, in load_plugins_by_group
ERROR 10-04 21:18:16 [__init__.py:57]     func = plugin.load()
ERROR 10-04 21:18:16 [__init__.py:57]   File "/usr/lib/python3.10/importlib/metadata/__init__.py", line 171, in load
ERROR 10-04 21:18:16 [__init__.py:57]     module = import_module(match.group('module'))
ERROR 10-04 21:18:16 [__init__.py:57]   File "/usr/lib/python3.10/importlib/__init__.py", line 126, in import_modul

RuntimeError: Device string must not be empty